# GEOS 5314 Exercise 11: Introduction to Downscaling (local bias correction)

This notebook gives an introduction into local bias correction, generally referred to as "statistical downscaling".  Intended to run in custon conda environment *geoClimate*.  If loading additional packages, be sure they have been installed using "conda install -c conda-forge packagename".
This Notebook is licensed for free and open consumption under the [Attribution-NonCommercial 4.0 International (CC BY-NC 4.0)](https://creativecommons.org/licenses/by-nc/4.0/) license.

CVS: $Id: Downscaling.ipynb,v 25.4 2025/04/08 01:18:56 brikowi Exp $

In [ ]:
# Load needed packages
import glob
import matplotlib.pyplot as plt
import urllib.request              # Manage URL's
import numpy as np                 # Basic math and simple arrays
import xarray as xr                # Better & faster multidimensional arrays
import zarr                        # Chunked very fast cloud-based arrays
from sklearn.linear_model import LinearRegression  # Use conda install -c conda-forge scikit-learn
from sklearn.metrics import mean_squared_error


import pandas as pd                # Dataframes, similar to internal spreadsheet
from difflib import get_close_matches
import gcsfs                       # Google cloud filesystem routines
import datetime
import os
import random

## Get list of models available at Google
Get file list from Google via the given URL, use Pandas (pd) to read the CSV file and save in a *dataframe* "df".  A Pandas dataframe is like a spreadsheet format for Python.

In [ ]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')

## Get list of all available SSP3 7.0 models with *tasmax*
Find lines in the file-list dataframe "df" that meet the given criteria, most important are that the variable *tasmax* is in the file and that it is from a model of experiment *ssp370* (the currently most-likely CO2 emissions scenario).

In [ ]:
df_ssp370 = df.query("activity_id=='ScenarioMIP' & table_id == 'Amon' & " +\
    "variable_id == 'tasmax' & experiment_id == 'ssp370' & member_id == 'r1i1p1f1'")
print(len(df_ssp370), 'forecast files match the search criteria, the first 3 are:')
df_ssp370.head(3)

## Get list of all historical models with *tasmax*

In [ ]:
df_historical = df.query("activity_id == 'CMIP' & table_id == 'Amon' & " +\
    "variable_id == 'tasmax' & experiment_id == 'historical' & member_id == 'r1i1p1f1'")
print(len(df_historical), 'historical files match the search criteria, the first 3 are:')
df_historical.head(3)

## Make list of models with BOTH historical and future tasmax
See <a href="https://stackoverflow.com/questions/55898796/how-to-match-keys-of-2-data-frames-and-create-new-df-with-matching-keys">StackOverflow</a> for suggestions.  Current code requires start with shortest dataframe (!).

In [ ]:
seq = [r for r in df_ssp370["source_id"] if get_close_matches(r, df_historical["source_id"], n=1, cutoff = .85)]
bothSources = np.unique(np.array(seq))
print(seq)

print(len(bothSources), "models have both historical and SSP3-7.0 results")
print("e.g. ", bothSources[3])

gcs = gcsfs.GCSFileSystem(token='anon')


In [ ]:
for i in range(len(bothSources)):    # model = 'GFDL-CM4'
    print("Model %d : %s" % (i, bothSources[i]))


## Define functions to get desired variable from desired model

In [ ]:
def getVar (modelList, modelName):
    # Custom function to get variable for model 'modelName' from list of possible models 'modelList'
    zstore = modelList.query(f"source_id == '{modelName}'").zstore.values[0]
    ds = xr.open_zarr(zstore, consolidated = True)
    ds.load()
    return ds

In [ ]:
# Define lat-long point for evaluation, DFW is default here
locName = 'DFW'                     # Name of location (for labeling plots)
pointLat = 32.7666
pointLon = 360 - 96.7778   # Longitude must be positive!

# Compute RMSE of historical model vs. PRISM-observed T<sub>max</sub>


## Read local observed T<sub>max</sub> from downloaded PRISM file

Use the file for you local area downloaded from [PRISM explorer](https://prism.oregonstate.edu/explorer/).  Rename that file '*PRISMobserved_1950-2014.csv*' (or change the variable *PRISMfile* in the next cell), and be sure the last header line contains '*Date*' and '*tmean (degrees C)*'.  The file must be in the directory from which you started this Notebook, or give the full path to the file in *PRISMfile*.

In [ ]:
PRISMfile = "PRISMobserved_1950-2014.csv"
# PRISMfile = "/home/brikowi/Teaching/Resilience/Exercises/Downscaling/PRISM_ppt_tmean_tmax_stable_4km_195001_201412_32.7666_-96.7778.csv"
obsDF = pd.read_csv(PRISMfile, skiprows=10)
if (len(obsDF) != 780):
    print("PRISM file needs monthly data for years 1950-2014, i.e. 780 entries.  Your file has %d entries" %
         len(obsDF) )
    print("Aborting.  Please provide the correct file")
obsDF = obsDF.rename(columns={'tmean (degrees C)': 'tmean', 'tmax (degrees C)': 'tmax'})
obsDF

## Compute 100 quantiles of observed T<sub>max</sub>
These are used to force (downscale for bias removal) the historical model T<sub>max</sub> to have the same distribution about the median T<sub>max</sub>.

In [ ]:
obs_quantiles = obsDF.tmax.quantile(np.arange(0.01, 1, 0.01))
obs_quantiles

### Choose a single model for downscaling at your selected point
Model chosen at random from those with historical and future results for ssp370

In [ ]:
%%time
# model = bothSources[random.randint(0,len(bothSources))]      # Choose random model
model = 'GFDL-ESM4'                                            # Select model with known serious bias
print("Processing model: %s" % (model), end='...')
ds_hist = getVar(df_historical, model)

### Adjust times to be consistent between the datasets, for proper interpolation

In [ ]:
start_time = pd.to_datetime(datetime.date(1850,1,15)) # I chose 15 for all dates to make it easier.
time_new_hist = [start_time + pd.DateOffset(months = x) for x in range(len(ds_hist.time))]
ds_hist = ds_hist.assign_coords(time = time_new_hist)

start_date = pd.to_datetime(datetime.date(1950,1,1))
ds_hist_sel = ds_hist.isel(time=(ds_hist.time >= start_date))
pointHist = ds_hist_sel.sel(lat=pointLat, lon=pointLon, method="nearest").tasmax - 273.15 # Convert to Celsius
histDF = pointHist.to_dataframe().drop({'height','lat','lon'}, axis=1)

Slightly trickier for the observed dataset, which doesn't have CMIP-compatible dates

In [ ]:
xObs = xr.DataArray(obsDF.tmax.values,coords={'time':pointHist["time"].values},
                    dims=['time'])

### Compute same quantiles for the historical model

In [ ]:
hist_quantiles = histDF.tasmax.quantile(np.arange(0.01, 1, 0.01))
hist_quantiles

# Make linear regression between the model and observed T<sub>max</sub>
This is *Quantile Mapping*, the simplest of the downscaling methods (for bias, AKA error adustment).  We use linear regression to derive an equation that adjusts the modeled quantiles to agree with historical modeled quantiles.  The regression line plotted below is effectively a 1:1 fit between those two sets of quantiles.   

In [ ]:
# Fit a linear regression between the quantiles of the observed and simulated data
reg = LinearRegression()
reg.fit(hist_quantiles.to_frame(), obs_quantiles.to_frame())
# Resorted to extreme treatment to avoid error
#  DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. 
#  Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
slope = reg.coef_[0].squeeze().item()
intercept = reg.intercept_[0]
print("The regression equation is: downscaled_Tmax = %7.4f*modeled_Tmax + %7.4f" % (slope, intercept))

## Plot the regression

In [ ]:
plt.scatter(obs_quantiles.to_frame(), hist_quantiles.to_frame(),  color='black')
plt.plot(hist_quantiles.to_frame(), reg.predict(hist_quantiles.to_frame()), color='blue', linewidth=3, label="Regression")
plt.legend(fontsize=13)
plt.xticks(fontsize=12)
plt.yticks(fontsize=13)
plt.ylabel('Historical Model Tmax', fontsize=13, fontweight='bold')
plt.xlabel('Observed Tmax (degC)', fontsize=13, fontweight='bold')
plt.title(f'Quantile Mapping, Historical vs. Observed Tmax: {model}',
          fontsize=15)

plt.tight_layout()

# Compute downscaled T<sub>max</sub>
This is really just bias correction at a single lat-long point.  The black dots above are forced onto the blue line by the regression formula (reg.predict).  Complete statistical downscaling would interpolate coarse GCM output to a finer grid then apply this or similar technique at each point.

In [ ]:
# Downscale the data (output is numpy.array)
downscaled_data = reg.predict(histDF)

## Compute RMSE 
Use simple machine learning tools to see if model fit was improved by bias correction.

In [ ]:
raw_rmse  = mean_squared_error(y_true=obsDF.tmax, y_pred=histDF)
downscaled_rmse = mean_squared_error(y_true=histDF, y_pred=downscaled_data)
print("RMSE between historical model and observed: %7.4f, and downscaled and observed: %7.4f" % 
      (raw_rmse, downscaled_rmse))

# Plot the datasets
Observed (PRISM), uncorrected GCM historical model, and downscaled (bias-corrected) model

In [ ]:
# Convert GCM and Downscaled to xarray for export to file and/or plotting
xGCM = histDF.to_xarray()
xDScal = xr.DataArray(downscaled_data.reshape(len(downscaled_data)), coords={'time':histDF.index}, 
                      dims={"time"})

In [ ]:
# Assemble as Pandas dataframe aggregated and indexed by year for easy export. Probably not the most efficient
tempX1 = xGCM.groupby('time.year').mean().tasmax
pdGCM = tempX1.to_pandas()
tempX2 = xDScal.groupby('time.year').mean()
pdDownscale = tempX2.to_pandas()
tempX3 = xObs.groupby('time.year').mean()
pdPRISM = tempX3.to_pandas()
frames = [pdGCM, pdDownscale, pdPRISM]
toExport = pd.concat(frames, axis=1)

toExport.rename(columns={'tasmax': 'rawGCM', 0: 'downscaleGCM', 1: 'PRISM'}, inplace=True)
toExport.to_csv('downscalingPlotData.csv', index=True)
